## Assignment 2.2 Sourcing data with an API

In [3]:
## Import and concatenate

In [5]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime

In [7]:
%%capture
import warnings
warnings.filterwarnings("ignore")

### As a strategic approach, since the data is too big, every second month of the each quarter's first trip data taken into account.

In [10]:
# Create a list with all files in the folder using a list compehension
import glob

filepaths = glob.glob("Data/*.csv")  # Only select CSV files
filepaths

['Data\\202201-citibike-tripdata_1.csv',
 'Data\\202202-citibike-tripdata_1.csv',
 'Data\\202203-citibike-tripdata_1.csv',
 'Data\\202204-citibike-tripdata_1.csv',
 'Data\\202205-citibike-tripdata_1.csv',
 'Data\\202206-citibike-tripdata_1.csv',
 'Data\\202207-citibike-tripdata_1.csv',
 'Data\\202208-citibike-tripdata_1.csv',
 'Data\\202209-citibike-tripdata_1.csv',
 'Data\\202210-citibike-tripdata_1.csv',
 'Data\\202211-citibike-tripdata_1.csv',
 'Data\\202212-citibike-tripdata_1.csv']

### We used a generator to list comprehensions, since we need to iterate through the files just once, and generators are capable of handling one file at a time

In [13]:
df = pd.concat((pd.read_csv(f).sample(n=min(100000, len(pd.read_csv(f))), random_state=42) for f in filepaths), ignore_index=True)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C31A06A81423A14C,classic_bike,2022-01-18 07:40:25.346,2022-01-18 07:43:35.441,W 26 St & 8 Ave,6297.02,10 Ave & W 28 St,6459.04,40.747348,-73.997236,40.750664,-74.001768,member
1,6104B30E6CC08F21,classic_bike,2022-01-18 20:13:23.444,2022-01-18 20:57:37.652,Jay St & York St,4895.09,Clark St & Henry St,4789.03,40.701403,-73.986727,40.697601,-73.993446,member
2,127F7DF7F787F370,classic_bike,2022-01-01 02:26:22.776,2022-01-01 02:46:12.503,Greenwich St & Perry St,5922.04,E 13 St & Avenue A,5779.09,40.734980,-74.006970,40.729667,-73.980680,casual
3,FF12FDCC260D47C3,classic_bike,2022-01-06 18:30:28.542,2022-01-06 18:44:35.763,Lexington Ave & E 36 St,6313.1,W 26 St & 10 Ave,6382.05,40.747574,-73.978801,40.749718,-74.002950,member
4,D8402474AF50D5F1,classic_bike,2022-01-27 18:29:15.544,2022-01-27 18:30:52.077,31 St & 21 Ave,7202.07,31 St & Ditmars Blvd,7144.02,40.778130,-73.908420,40.776168,-73.910485,member


### We identify the available csv files with the data path. Then we went over the loop to read for each csv file row by row and concatenated it.

In [15]:
df.shape

(1200000, 13)

In [16]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C31A06A81423A14C,classic_bike,2022-01-18 07:40:25.346,2022-01-18 07:43:35.441,W 26 St & 8 Ave,6297.02,10 Ave & W 28 St,6459.04,40.747348,-73.997236,40.750664,-74.001768,member
1,6104B30E6CC08F21,classic_bike,2022-01-18 20:13:23.444,2022-01-18 20:57:37.652,Jay St & York St,4895.09,Clark St & Henry St,4789.03,40.701403,-73.986727,40.697601,-73.993446,member
2,127F7DF7F787F370,classic_bike,2022-01-01 02:26:22.776,2022-01-01 02:46:12.503,Greenwich St & Perry St,5922.04,E 13 St & Avenue A,5779.09,40.734980,-74.006970,40.729667,-73.980680,casual
3,FF12FDCC260D47C3,classic_bike,2022-01-06 18:30:28.542,2022-01-06 18:44:35.763,Lexington Ave & E 36 St,6313.1,W 26 St & 10 Ave,6382.05,40.747574,-73.978801,40.749718,-74.002950,member
4,D8402474AF50D5F1,classic_bike,2022-01-27 18:29:15.544,2022-01-27 18:30:52.077,31 St & 21 Ave,7202.07,31 St & Ditmars Blvd,7144.02,40.778130,-73.908420,40.776168,-73.910485,member


In [17]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
1199995,E45C259260059D62,classic_bike,2022-12-10 11:18:51.279,2022-12-10 11:23:10.473,E 13 St & Avenue A,5779.09,E 11 St & 3 Ave,5788.16,40.729667,-73.980680,40.731270,-73.988490,member
1199996,61C69C13D6AD2130,electric_bike,2022-12-13 11:59:47.939,2022-12-13 12:05:46.776,Lewis Ave & Kosciuszko St,4617.01,Broadway & Hancock St,4441.06,40.692391,-73.937022,40.686630,-73.916800,member
1199997,1DDC6329C0FF397D,electric_bike,2022-12-22 04:13:23.684,2022-12-22 04:44:50.778,E 163 St & Union Ave,7970.05,E 33 St & 1 Ave,6197.08,40.822507,-73.901631,40.743227,-73.974498,member
1199998,941720ACE7ACC139,electric_bike,2022-12-19 17:15:08.615,2022-12-19 17:23:54.069,W 84 St & Columbus Ave,7382.04,Frederick Douglass Blvd & W 112 St,7631.23,40.785000,-73.972834,40.801694,-73.957145,member
1199999,5140115C179ACA5D,electric_bike,2022-12-28 11:58:49.372,2022-12-28 12:11:19.813,Church Ave & McDonald Ave,3090.06,5 Ave & 37 St,3379.03,40.642894,-73.979087,40.652657,-74.002356,casual


### Heads and Tails match, so they merged perfectly

In [19]:
df = pd.concat(
    (pd.read_csv(f, dtype={"start_station_name": str, "end_station_name": str}, low_memory=False) for f in filepaths),
    ignore_index=True
)


In [20]:
print(df.dtypes)  # Should show 'object' for start_station_name and end_station_name
print(df[['start_station_name', 'end_station_name']].head())  # Preview data


ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object
        start_station_name             end_station_name
0  West End Ave & W 107 St  Mt Morris Park W & W 120 St
1             4 Ave & 3 St      Boerum Pl\t& Pacific St
2          1 Ave & E 62 St              5 Ave & E 29 St
3          2 Ave & E 96 St              5 Ave & E 29 St
4          6 Ave & W 34 St              5 Ave & E 29 St


In [21]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member


In [22]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
11999995,2C18D66E198EA410,classic_bike,2022-12-04 20:11:00.788,2022-12-04 20:19:15.088,6 Ave & W 33 St,6364.07,9 Ave & W 39 St,6644.08,40.749013,-73.988484,40.756404,-73.994101,member
11999996,3C0686D88E2F6AB5,classic_bike,2022-12-14 18:51:45.520,2022-12-14 18:57:44.283,E 20 St & Park Ave,6055.08,Ave A & E 14 St,5779.11,40.738274,-73.987520,40.730311,-73.980472,member
11999997,42AAD1775F1D3138,classic_bike,2022-12-31 18:13:19.297,2022-12-31 18:19:11.314,E 20 St & Park Ave,6055.08,Ave A & E 14 St,5779.11,40.738274,-73.987520,40.730311,-73.980472,member
11999998,DA4C5D66E1C60D25,classic_bike,2022-12-04 23:51:05.474,2022-12-05 00:20:25.181,Vesey St & Church St,5216.06,W 45 St & 8 Ave,6676.02,40.712220,-74.010472,40.759291,-73.988597,member
11999999,A570DB28B3868263,electric_bike,2022-12-23 17:46:18.594,2022-12-23 17:51:21.043,E 20 St & Park Ave,6055.08,E 11 St & 1 Ave,5746.14,40.738039,-73.987402,40.729538,-73.984267,member


## ## Get weather data using NOAA's API

In [24]:
# Define your NOAA token - type yours in here! 

Token = 'GCWhOUMGHMLILXunaHDbUZACXUbnFrOa' 
stationid="GHCND:USW00014732"

In [25]:
# Get the API 

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [26]:
# Load the api response as a json

d = json.loads(r.text)

In [27]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [28]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [29]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [30]:
temps[10:20]

[-54, -19, 40, 48, -67, -80, 39, 18, 32, 51]

In [31]:
# Put the results in a dataframe

df_temp = pd.DataFrame()

In [32]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [33]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [34]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [35]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [36]:
df['started_at'] = pd.to_datetime(df['started_at'], dayfirst = True)

In [37]:
df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [38]:
df['date'] = pd.to_datetime(df['date'])

In [39]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [40]:
%%time
df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)

CPU times: total: 11.2 s
Wall time: 11.4 s


In [41]:
df_merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member,2022-01-21,-6.0,both
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member,2022-01-10,1.6,both
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member,2022-01-26,-2.3,both
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member,2022-01-03,1.4,both
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member,2022-01-22,-5.9,both


### Average temperature and start time and date are merged.

In [43]:
df_merged['_merge'].value_counts(dropna = False)

both          11999379
left_only          621
right_only           0
Name: _merge, dtype: int64

In [44]:
df_merged.to_csv('Data/output/newyork_data.csv', index = False)

In [45]:
df.shape

(12000000, 14)

In [46]:
df_temp.to_csv('Data/output/nytemperature.csv', index = False)